In [1]:
import os
from typing import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


sns.set_style("whitegrid")

In [2]:
from dataclasses import dataclass, field
from glob import glob

import torch


@dataclass
class InputPath:
    _prefix: str = "../input"
    train: str = f"{_prefix}/train.csv"
    materials: str = f"{_prefix}/materials.csv"
    techniques: str = f"{_prefix}/techniques.csv"
    test: str = f"{_prefix}/test.csv"
    sub: str = f"{_prefix}/atmaCup#11_sample_submission.csv"
    photos_prefix: str = f"{_prefix}/photos"
    photos: List[str] = field(default_factory=lambda: glob(f"../input/photos/*.jpg"))
        
        
@dataclass
class OutputPath:
    _prefix: str = "../output/"
    model: str = f"{_prefix}/model"
    submission: str = f"{_prefix}/submission"

        
@dataclass
class Basic:
    run_name: str = "exp001"
    is_debug: bool = False
    seed: int = 42
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
        
        
@dataclass
class Kfold:
    number: int = 5
    method: str = "skf"
    shuffle: bool = True
    columns: List[str] = field(default_factory=lambda: ["target"])
        
        
@dataclass
class Adam:
    name: str = "Adam"
    lr: float = 1e-4
    weight_decay: float = 0
    amsgrad: bool = False
        
        
@dataclass
class ReduceLROnPlateau:
    name: str = "ReduceLROnPlateau"
    mode: str = "min"
    factor: float = 0.1
    patience: int = 5
    verbose: bool = True
    eps: float = 1e-8

        
@dataclass
class Params:
    batch_size: int = 64
    epochs: int = 3 if Basic.is_debug else 10
    image_size: int = 224
    max_grad_norm: int = 1000
    num_workers: int = 0
    print_freq: int = 10000
    target_size: int = 1
    # Union[Adam]
    optimizer: Adam = Adam()
    # Union[CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau]
    scheduler: ReduceLROnPlateau = ReduceLROnPlateau()
    pretrained: bool = False
        

@dataclass
class ModelConfig:
    basic: Basic = Basic()
    kfold: Kfold = Kfold()
    params: Params = Params()

In [3]:
import math
import os
import random
import time

import joblib
import numpy as np
import torch
from sklearn import metrics


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class Jbl:
    @staticmethod
    def load(filepath: str) -> Any:
        return joblib.load(filepath)

    @staticmethod
    def save(obj_: Any, filepath: str) -> None:
        joblib.dump(obj_, filepath, compress=3)


def RMSE(y: np.array, p: np.array) -> float:
    return metrics.mean_squared_error(y, p) ** 0.5


def fix_seed(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
    
def time_since(since: time.time, percent: float) -> str:
    def as_minutes(s):
        m = math.floor(s / 60)
        s -= m * 60
        return "%dm %ds" % (m, s)

    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (as_minutes(s), as_minutes(rs))

In [4]:
from PIL import Image


def to_img_path(photo_dir: str, object_id: str) -> str:
    return os.path.join(photo_dir, f'{object_id}.jpg')

def read_image(object_id: str):
    return Image.open(to_img_path(object_id))

In [5]:
import pandas as pd


def load_csv(path: str) -> pd.DataFrame:
    return pd.read_csv(path)

In [6]:
from sklearn.model_selection import (
    GroupKFold,
    KFold,
    StratifiedKFold,
    TimeSeriesSplit,
)


def generate_kf(cfg: ModelConfig) -> Generator:
    if cfg.kfold.method == "kf":
        kf = KFold(
            n_splits=cfg.kfold.number,
            shuffle=cfg.kfold.shuffle,
            random_state=cfg.basic.seed,
        )
    elif cfg.kfold.method == "skf":
        kf = StratifiedKFold(
            n_splits=cfg.kfold.number,
            shuffle=cfg.kfold.shuffle,
            random_state=cfg.basic.seed,
        )
    elif cfg.kfold.method == "gkf":
        kf = GroupKFold(n_splits=cfg.kfold.number)
    elif cfg.kfold.method == "sgkf":
        kf = StratifiedGroupKFold(
            n_splits=cfg.kfold.number, random_state=cfg.basic.seed
        )
    elif cfg.kfold.method == "tskf":
        kf = TimeSeriesSplit(n_splits=cfg.kfold.number)
    else:
        raise ValueError(f"{cfg.kfold.method} is not supported")
    return kf

In [7]:
import pandas as pd
from PIL import Image
from torch.utils import data
from torchvision import transforms as T


IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]


class AtmaDataset(data.Dataset):
    """atmaCup用にデータ読み込み等を行なうデータ・セット"""
    object_path_key = "object_path"
    label_key = "target"

    def __init__(self, meta_df: pd.DataFrame, is_train: bool = True) -> None:
        self.is_train = is_train
        self.meta_df = meta_df.reset_index(drop=True)
        self.index_to_data = self.meta_df.to_dict(orient="index")
        
        size = (ModelConfig.params.image_size, ModelConfig.params.image_size)
        additional_items = (
            [T.Resize(size)]
            if not is_train
            else [
                T.RandomGrayscale(p=0.2),
                T.RandomHorizontalFlip(),
                T.ColorJitter(
                    brightness=0.3,
                    contrast=0.5,
                    saturation=[0.8, 1.3],
                    hue=[-0.05, 0.05],
                ),
                T.RandomResizedCrop(size),
            ]
        )
        self.transformer = T.Compose(
            [*additional_items, T.ToTensor(), T.Normalize(mean=IMG_MEAN, std=IMG_STD)]
        )
        
        self._validate_meta_df(meta_df)

    def __getitem__(self, index) -> Dict[str, Any]:
        data = self.index_to_data[index]
        obj_path, label = data.get(self.object_path_key), data.get(self.label_key, -1)
        img = self.transformer(Image.open(obj_path))
        return {"image": img, "label": label}

    def __len__(self) -> int:
        return len(self.meta_df)
    
    def _validate_meta_df(self, meta_df: pd.DataFrame) -> None:
        for k in self.meta_keys:
            if k not in meta_df:
                raise ValueError(f"meta df must have {k}")
                
    @property
    def meta_keys(self) -> List[str]:
        retval = [self.object_path_key]
        if self.is_train:
            retval += [self.label_key]
        return retval

In [8]:
import time
from collections import OrderedDict
from logging import Logger
from typing import Dict, Union

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import models
from tqdm import tqdm


def build_model(pretrained: bool = False):
    model = models.resnet34(pretrained=pretrained)
    model.fc = nn.Linear(in_features=512, out_features=1, bias=True)    
    return model


class BaseRunner:
    def __init__(self, cfg: ModelConfig):
        self.cfg = cfg
        self.params = cfg.params

    def _get_scheduler(
        self, optimizer: Union[optim.Adam]
    ) -> Union[
        lr_scheduler.ReduceLROnPlateau,
    ]:
        if self.params.scheduler.name == "ReduceLROnPlateau":
            scheduler = lr_scheduler.ReduceLROnPlateau(
                optimizer,
                mode=self.params.scheduler.mode,
                factor=self.params.scheduler.factor,
                patience=self.params.scheduler.patience,
                verbose=self.params.scheduler.verbose,
                eps=self.params.scheduler.eps,
            )
        else:
            raise ValueError(f"{self.params.scheduler.name} is not supported")
        return scheduler

    def _step_scheduler(
        self,
        scheduler: Union[
            lr_scheduler.ReduceLROnPlateau,
        ],
        avg_val_loss,
    ) -> Union[
        lr_scheduler.ReduceLROnPlateau,
    ]:
        if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        else:
            raise ValueError(f"{self.params.shceduler.name} is not supported")
        return scheduler

    def _evaluate(self, y_true: np.array, y_pred: np.array):
        score = RMSE(y_true, y_pred)
        print(f"Score: {score:<.5f}")


class TrainRunner(BaseRunner):
    def _train_batch(self, train_loader, model, criterion, optimizer, scheduler, epoch):
        batch_time = AverageMeter()
        data_time = AverageMeter()
        losses = AverageMeter()
        # scores = AverageMeter()
        model.train()
        start = end = time.time()
        for step, image_label_dict in enumerate(train_loader):
            data_time.update(time.time() - end)

            images = image_label_dict.get("image").to(self.cfg.basic.device)
            labels = image_label_dict.get("label").to(self.cfg.basic.device)
            batch_size = labels.size(0)

            y_preds = model(images)
            loss = criterion(y_preds, labels.float())
            losses.update(loss.item(), batch_size)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            grad_norm = nn.utils.clip_grad_norm_(
                model.parameters(), self.params.max_grad_norm
            )
            batch_time.update(time.time() - end)
            end = time.time()
#             if step % self.params.print_freq == 0 or step == (len(train_loader) - 1):
#                 print(
#                     f"""
#                     Epoch: [{epoch + 1}][{step}/{len(train_loader)}]
#                     Batch {batch_time.val:.3f} ({batch_time.avg:.3f})
#                     Data {data_time.val:.3f} ({data_time.avg:.3f})
#                     Elapsed {time_since(start, float(step + 1) / len(train_loader)):s}
#                     Loss: {losses.val:.4f}({losses.avg:.4f})
#                     Grad: {grad_norm:.4f}
#                     """
#                 )
        return losses.avg

    def _valid_batch(self, valid_loader, model, criterion):
        batch_time = AverageMeter()
        data_time = AverageMeter()
        losses = AverageMeter()
        # scores = AverageMeter()
        model.eval()
        preds = []
        start = end = time.time()
        for step, image_label_dict in enumerate(valid_loader):
            data_time.update(time.time() - end)

            images = image_label_dict.get("image").to(self.cfg.basic.device)
            labels = image_label_dict.get("label").to(self.cfg.basic.device)
            batch_size = labels.size(0)

            with torch.no_grad():
                y_preds = model(images)
            loss = criterion(y_preds, labels.float())
            losses.update(loss.item(), batch_size)
            preds.append(y_preds.to("cpu").numpy())

            batch_time.update(time.time() - end)
            end = time.time()
#             if step % self.params.print_freq == 0 or step == (len(valid_loader) - 1):
#                 print(
#                     f"""
#                     EVAL: [{step}/{len(valid_loader)}]
#                     Batch {batch_time.val:.3f} ({batch_time.avg:.3f})
#                     Data {data_time.val:.3f} ({data_time.avg:.3f})
#                     Elapsed {time_since(start, float(step + 1) / len(valid_loader)):s}
#                     Loss: {losses.val:.4f}({losses.avg:.4f})
#                     """
#                 )
        predictions = np.concatenate(preds)
        return losses.avg, predictions

    def _train(self, train: pd.DataFrame, n_fold: int):
        print(f"fold: {n_fold}")

        trn_idx = train[train["fold"] != n_fold].index
        val_idx = train[train["fold"] == n_fold].index
        train_folds = train.loc[trn_idx].reset_index(drop=True)
        valid_folds = train.loc[val_idx].reset_index(drop=True)
        train_dataset = AtmaDataset(
            train_folds,
            is_train=True,
#             transform=get_transforms(self.params, data="train"),
        )
        valid_dataset = AtmaDataset(
            valid_folds,
            is_train=False,
#             transform=get_transforms(self.params, data="valid"),
        )
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.params.batch_size,
            shuffle=True,
            num_workers=self.params.num_workers,
            pin_memory=True,
            drop_last=True,
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=self.params.batch_size,
            shuffle=False,
            num_workers=self.params.num_workers,
            pin_memory=True,
            drop_last=False,
        )

        model = build_model(pretrained=self.cfg.params.pretrained)
        model.to(self.cfg.basic.device)
        optimizer = optim.Adam(
            model.parameters(),
            lr=self.params.optimizer.lr,
            weight_decay=self.params.optimizer.weight_decay,
            amsgrad=self.params.optimizer.amsgrad,
        )
        scheduler = self._get_scheduler(optimizer)

        criterion = nn.MSELoss()
        best_score = np.inf
        # best_loss = np.inf
        for epoch in range(self.params.epochs):
            start_time = time.time()

            avg_loss = self._train_batch(
                train_loader, model, criterion, optimizer, scheduler, epoch
            )
            avg_val_loss, preds = self._valid_batch(valid_loader, model, criterion)
            valid_labels = valid_folds["target"].values
            scheduler = self._step_scheduler(scheduler, avg_val_loss)
            score = RMSE(valid_labels, preds)

            elapsed = time.time() - start_time
            print(
                f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
            )
            print(f"Epoch {epoch+1} - RMSE: {score}")
            if best_score > score:
                best_score = score
                torch.save(
                    {"model": model.state_dict(), "preds": preds},
                    f"{OutputPath.model}/{self.cfg.basic.run_name}_{n_fold}.pth",
                )
            print(
                f"Epoch {epoch+1} - Best Score: {best_score:.4f}"
            )

        check_point: Dict[str, Union[OrderedDict, torch.Tensor]] = torch.load(
            f"{OutputPath.model}/{self.cfg.basic.run_name}_{n_fold}.pth"
        )
        valid_folds["preds"] = check_point["preds"]
        return valid_folds

    def run_cv(self, train: pd.DataFrame) -> None:
        oof_df = pd.DataFrame()
        for n_fold in range(self.cfg.kfold.number):
            _oof_df = self._train(train, n_fold)
            print(f"========== fold: {n_fold} result ==========")
            self._evaluate(_oof_df["target"], _oof_df["preds"])
            oof_df = pd.concat([oof_df, _oof_df])
        print("========== CV ==========")
        self._evaluate(oof_df["target"], oof_df["preds"])
        Jbl.save(oof_df, f"{OutputPath.model}/oof_df_{self.cfg.basic.run_name}.jbl")

In [9]:
fix_seed()
input_path = InputPath()
model_config = ModelConfig()

In [10]:
train = load_csv(input_path.train)
train = train.assign(object_path = train["object_id"].apply(lambda x: to_img_path(input_path.photos_prefix, x)))
print(train.shape)
display(train.head(3))

(3937, 5)


,object_id,sorting_date,art_series_id,target,object_path
0,002bff09b09998d0be65,1631,509357f67692a6a45626,1,../input/photos/002bff09b09998d0be65.jpg
1,00309fb1ef05416f9c1f,1900,7987b47bbe5dc3039179,3,../input/photos/00309fb1ef05416f9c1f.jpg
2,003a1562e97f79ba96dc,1834,ded7c3c9636708e5b14c,3,../input/photos/003a1562e97f79ba96dc.jpg


In [11]:
kf = generate_kf(model_config)
if model_config.kfold.method.endswith("gkf"):
    kf_generator = kf.split(
        train,
        train[self.fe_cfg.column.target],
        groups=train[self.kfold.columns],
    )
else:
    kf_generator = kf.split(train, train["target"])
for fold_i, (tr_idx, val_idx) in enumerate(kf_generator):
    print(
        f"fold: {fold_i} - {tr_idx[:5]}~{tr_idx[-5:]}\t{val_idx[:5]}~{val_idx[-5:]}"
    )
    train.loc[val_idx, "fold"] = fold_i
train = train.assign(fold = train["fold"].astype(int))
if model_config.kfold.method == "skf":
    display(train.groupby(["fold", "target"]).size().to_frame())

fold: 0 - [0 1 2 3 4]~[3928 3930 3933 3934 3935]	[ 8 14 16 19 21]~[3922 3929 3931 3932 3936]
fold: 1 - [0 1 2 4 5]~[3931 3932 3933 3934 3936]	[ 3  9 20 23 24]~[3899 3907 3912 3925 3935]
fold: 2 - [0 1 2 3 5]~[3931 3932 3934 3935 3936]	[ 4  6 10 11 12]~[3913 3915 3924 3927 3933]
fold: 3 - [2 3 4 5 6]~[3932 3933 3934 3935 3936]	[ 0  1  7 17 29]~[3914 3916 3919 3923 3926]
fold: 4 - [0 1 3 4 6]~[3931 3932 3933 3935 3936]	[ 2  5 22 31 32]~[3909 3918 3928 3930 3934]


0
fold target     
0    0        95
     1       180
     2       302
     3       211
1    0        95
     1       179
     2       303
     3       211
2    0        95
     1       179
     2       302
     3       211
3    0        95
     1       179
     2       302
     3       211
4    0        95
     1       179
     2       302
     3       211

In [12]:
TrainRunner(model_config).run_cv(train)

fold: 0


/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 - avg_train_loss: 1.1204  avg_val_loss: 0.9420  time: 23s
Epoch 1 - RMSE: 0.9695065706379833
Epoch 1 - Best Score: 0.9695


/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2 - avg_train_loss: 0.9460  avg_val_loss: 0.9559  time: 23s
Epoch 2 - RMSE: 0.9777480897695329
Epoch 2 - Best Score: 0.9695


...

/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([19])) that is different to the input size (torch.Size([19, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 9 - avg_train_loss: 0.9504  avg_val_loss: 0.9449  time: 23s
Epoch 9 - RMSE: 0.9738104217367641
Epoch 9 - Best Score: 0.9692


/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 10 - avg_train_loss: 0.9491  avg_val_loss: 0.9416  time: 23s
Epoch 10 - RMSE: 0.9718088875236508
Epoch 10 - Best Score: 0.9692
========== fold: 4 result ==========
Score: 0.96918
========== CV ==========
Score: 0.96763


/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([19])) that is different to the input size (torch.Size([19, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [13]:
Jbl.load(f"{OutputPath.model}/oof_df_{model_config.basic.run_name}.jbl")

,object_id,sorting_date,art_series_id,target,object_path,fold,preds
0,00bf812ffe8a62d45661,1720,3bfd41016d864e3fd8b5,2,../input/photos/00bf812ffe8a62d45661.jpg,0,1.768609
1,0110115b8b6036d9ab3c,1741,baa4a20f0372d74dfc80,2,../input/photos/0110115b8b6036d9ab3c.jpg,0,1.741917
2,01273c00c46a84c50468,1757,51024cb4c6256ccce827,2,../input/photos/01273c00c46a84c50468.jpg,0,1.768760
3,0160840d9a4620b08fbd,1874,a56504999a8157a25d16,3,../input/photos/0160840d9a4620b08fbd.jpg,0,1.820476
4,0169c493b09c8758a1b3,1734,550c62dd363fea62581c,2,../input/photos/0169c493b09c8758a1b3.jpg,0,1.733444
...,...,...,...,...,...,...,...
782,fe4d591332b08ab81361,1600,0782dfafa355acaf888c,0,../input/photos/fe4d591332b08ab81361.jpg,4,1.668022
783,fecae40c488e4c8c5ba5,1600,9ac00be75c55b1653a94,0,../input/photos/fecae40c488e4c8c5ba5.jpg,4,1.855575
784,ff37540e22e1ef455368,1765,9971eebf0f583a5e51da,2,../input/photos/ff37540e22e1ef455368.jpg,4,1.780144
785,ff9124278e0f7086738f,1630,b4d508f53d16f7bcaf55,1,../input/photos/ff9124278e0f7086738f.jpg,4,1.802254
